In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import os
import pandas as pd

import scipy.cluster.hierarchy as shc
from scipy.cluster.hierarchy import linkage, fclusterdata

import seaborn as sns

from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import StandardScaler

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Read the data with SQL

In [ ]:
# Creating a SparkSession
spark = SparkSession.builder.getOrCreate()

# Specifying the path to the database file
db_file_path = "/dbfs/FileStore/tables/digisusti/2024-footprint/olist.db"

# Checking if the file 'olist.db' exists in the specified path
df_customers = spark.read.format("jdbc").option("url", f"jdbc:sqlite:{db_file_path}").option("dbtable", "customers").option("fetchsize", "1000").load()
df_orders = spark.read.format("jdbc").option("url", f"jdbc:sqlite:{db_file_path}").option("dbtable", "orders").option("fetchsize", "1000").load()
df_order_reviews = spark.read.format("jdbc").option("url", f"jdbc:sqlite:{db_file_path}").option("dbtable", "order_reviews").option("fetchsize", "1000").load()
df_order_items = spark.read.format("jdbc").option("url", f"jdbc:sqlite:{db_file_path}").option("dbtable", "order_items").option("fetchsize", "1000").load()
df_order_pymts = spark.read.format("jdbc").option("url", f"jdbc:sqlite:{db_file_path}").option("dbtable", "order_pymts").option("fetchsize", "1000").load()

# Registering the DataFrames as temporary views
df_customers.createOrReplaceTempView("customers_view")
df_orders.createOrReplaceTempView("orders_view")
df_order_reviews.createOrReplaceTempView("order_reviews_view")
df_order_items.createOrReplaceTempView("order_items_view")
df_order_pymts.createOrReplaceTempView("order_pymts_view")

In [ ]:
result = spark.sql("""
SELECT
    c.customer_unique_id,
    (datediff(date('2018-10-17'), MIN(o.order_purchase_timestamp)) + 1) AS seniority,
    COUNT(or_reviews.order_id) AS nb_reviews,
    AVG(or_reviews.review_score) AS average_review_score,
    COUNT(o.order_id) AS nb_orders,
    AVG(oi.price) AS average_basket,
    SUM(CASE WHEN o.order_status = 'canceled' THEN 1 ELSE 0 END) AS nb_canceled
FROM
    customers_view c
JOIN
    orders_view o ON c.customer_id = o.customer_id
LEFT JOIN
    order_reviews_view or_reviews ON o.order_id = or_reviews.order_id
JOIN
    order_items_view oi ON o.order_id = oi.order_id
GROUP BY
    c.customer_unique_id
""")


In [ ]:
# Displaying the result
result.show()

In [ ]:
raw_df = result.toPandas()

# Transform and select features
.......

# Build model

## Agglomerative

In [ ]:
memory_used = df.memory_usage().sum()
print(memory_used)

In [ ]:
# Create an instance of AgglomerativeClustering with 3 clusters
clustering = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')

df = df.sample(frac=0.4)

# Fit the clustering model using numeric features (excluding object type features)
clustering.fit(df.select_dtypes(exclude='object'))

# Assign the cluster labels to a new column 'cluster' in the DataFrame
df['cluster'] = clustering.labels_

## DBSCAN

In [ ]:
dbscan = DBSCAN(eps=0.5, min_samples=5)  # Vous pouvez ajuster eps et min_samples en fonction de vos données
dbscan.fit(df.select_dtypes(exclude='object'))

In [ ]:
# Créer une instance de UMAP
umap_model = umap.UMAP(n_neighbors=10, min_dist=0.1, metric='euclidean')

# Réduire les dimensions
data_2d = umap_model.fit_transform(df.select_dtypes(exclude='object'))

# Visualiser les données réduites
plt.scatter(data_2d[:, 0], data_2d[:, 1], c=dbscan.labels_)
plt.show()

# Evaluation
.........